# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおけるエージェントの実装とテストを行っています。コンペティションでは、参加者が制作したエージェントが「20の質問」ゲームをプレイし、正しいターゲットワードを推測する能力を競います。

## 主な問題
ノートブックは、エージェントが質問を行い、ターゲットワードを推測するという問題に取り組んでいます。このプロセスでは、エージェントはターンの種類（質問、推測、回答）に応じて適切な行動を選択しなければなりません。

## 手法とライブラリ
主に以下の手法とライブラリが使用されています：

1. **エージェントの定義**：
   - `simple_agent1`、`simple_agent2`、`simple_agent3`、`simple_agent4`という4つのシンプルなエージェントが定義されています。各エージェントは、異なる動物（アヒル、鳥、豚、牛）に関する質問を行い、その推測と回答を提供します。

2. **Kaggle Environmentsライブラリ**：
   - `kaggle_environments`ライブラリが用いられ、エージェントが対戦する「20の質問」環境がセットアップされています。`make`関数を使用して環境を作成し、ゲームの実行を可能にしています。

3. **ゲームの実行**：
   - `env.run()`を用いて、定義したエージェントを伴ったゲームの実行を行い、その結果を取得します。

4. **ゲームの可視化**：
   - `env.render()`を用いて、ゲームの進行を表示し、プレイの状態を視覚的に確認できるようにしています。

このノートブックは、シンプルな質問と推測機能を持つエージェントを用いて、20の質問ゲームの基礎的な実装を採用しており、今後の発展としてより複雑な戦略や大規模な言語モデル（LLM）を組み込むことが期待されます。

---


# 用語概説 
以下は、ノートブック内で使用されている専門用語に関する簡単な解説です。初心者向けの明快な説明を心がけました。

1. **エージェント**:
   - エージェントとは、特定のタスクを実行するプログラムやロジックのことを指します。このノートブックでは、プレイヤーとして質問をするエージェントと、それに答えるエージェントがいます。

2. **turnType**:
   - `turnType`は、ゲームの現在の状態やエージェントの役割を示します。具体的には、「ask」（質問する）、 「guess」（推測する）、 「answer」（回答する）という種類があります。これにより、エージェントがどのアクションを取るべきかが決まります。

3. **Kaggle環境**:
   - Kaggle環境は、Kaggleで利用するために構築された特定のシミュレーション環境のことです。この環境には、特定のゲームやタスクを実行するためのルールや設定が含まれています。このノートブックでは、「20の質問」ゲームの環境が作成されています。

4. **デバッグモード**:
   - デバッグモードは、プログラムのテストやエラー検出を容易にするための設定です。このモードでは、エラーや状態の詳細が表示され、問題の特定がしやすくなります。

5. **エピソード**:
   - エピソードは、エージェントたちがゲーム内で相互作用する一連の進行です。ゲームが始まり、ターゲットワードを推測し、ゲームが終了するまでの流れを指します。

6. **状態表示**:
   - `print(obs)`のように、状態表示は、現在のゲームの進行状況やエージェントの情報を出力することです。この情報はデバッグやエージェントの振る舞いを確認するために役立ちます。

7. **オブジェクト指向プログラミング (OOP)**:
   - Pythonのクラスやオブジェクトは、状態（データ）と振る舞い（メソッド）をまとめて管理するための手法です。このノートブックでは、特にエージェントの実装でこの手法が見られます。

これらの用語は、Kaggleの「20の質問」コンペティションやこのノートブックの文脈において専門的な意味を持ちますが、初心者にとってはあまり馴染みのない内容かもしれません。

---


In [ ]:
# このPython 3環境には、多くの役立つ分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下は読み込むのに役立ついくつかのパッケージです

import numpy as np # 線形代数を扱うライブラリ
import pandas as pd # データ処理、CSVファイルの入出力用ライブラリ (例: pd.read_csv)

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると（実行ボタンをクリックするか、Shift+Enterを押すことで）、入力ディレクトリ内のすべてのファイルがリスト表示されます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):  # 指定されたディレクトリ内のファイルを走査
    for filename in filenames:  # 各ファイルに対して
        print(os.path.join(dirname, filename))  # ファイルのフルパスを表示

# 現在のディレクトリ（/kaggle/working/）には最大20GBまで書き込むことができ、"Save & Run All"を使用してバージョンを作成すると出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、それらは現在のセッションの外では保存されません

In [ ]:
def simple_agent1(obs, cfg):    
    # エージェントが質問者の場合、turnTypeが"ask"のとき
    if obs.turnType == "ask": response = "Is it a duck?"  # 「それはアヒルですか？」と質問
    elif obs.turnType == "guess": response = "duck"  # 質問の答えとして「アヒル」と推測
    elif obs.turnType == "answer": response = "no"  # 「いいえ」と回答
    print(obs)  # 状態を表示
    return response  # 反応を返す

def simple_agent2(obs, cfg):
    # エージェントが質問者の場合、turnTypeが"ask"のとき
    if obs.turnType == "ask": response = "Is it a bird?"  # 「それは鳥ですか？」と質問
    elif obs.turnType == "guess": response = "bird"  # 質問の答えとして「鳥」と推測
    elif obs.turnType == "answer": response = "no"  # 「いいえ」と回答
    print(obs)  # 状態を表示
    return response  # 反応を返す

def simple_agent3(obs, cfg):
    # エージェントが質問者の場合、turnTypeが"ask"のとき
    if obs.turnType == "ask": response = "Is it a pig?"  # 「それは豚ですか？」と質問
    elif obs.turnType == "guess": response = "pig"  # 質問の答えとして「豚」と推測
    elif obs.turnType == "answer": response = "no"  # 「いいえ」と回答
    print(obs)  # 状態を表示
    return response  # 反応を返す

def simple_agent4(obs, cfg):
    # エージェントが質問者の場合、turnTypeが"ask"のとき
    if obs.turnType == "ask": response = "Is it a cow?"  # 「それは牛ですか？」と質問
    elif obs.turnType == "guess": response = "cow"  # 質問の答えとして「牛」と推測
    elif obs.turnType == "answer": response = "no"  # 「いいえ」と回答
    print(obs)  # 状態を表示
    return response  # 反応を返す

In [ ]:
from kaggle_environments import make  # Kaggle環境を作成するためのライブラリをインポート
env = make(environment="llm_20_questions", debug=True)  # 「20の質問」環境を作成し、デバッグモードを有効にする

In [ ]:
game_output = env.run(agents=[simple_agent1, simple_agent3, simple_agent2, simple_agent4])  # エージェントたち（simple_agent1、simple_agent3、simple_agent2、simple_agent4）を使用してゲームを実行し、その出力を取得する

In [ ]:
env.render(mode="ipython", width=400, height=400)  # ゲームの進行状況をIPython形式で表示し、幅と高さを400ピクセルに設定する

In [ ]:
# 何もコードが入力されていません。このセルには処理が必要な内容がありません。